In [4]:
import time
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import pickle
tf.__version__

'1.1.0'

### Import Model Inputs

In [6]:
# Load prompts
with open('kramer_source_list.pkl', 'rb') as file:
    source_int = pickle.load(file)

In [7]:
# Load responses (typo here intentional, was made in "Process_Scripts")
with open('kramer_reponse_list.pkl', 'rb') as file:
    target_int = pickle.load(file)

In [ ]:
# Load vocabulary
with open('kramer_vocab.pkl', 'rb') as file:
    vocab = pickle.load(file)

In [ ]:
# Load word to integer dictionary
with open('kramer_word2int.pkl', 'rb') as file:
    word2int = pickle.load(file)

In [ ]:
# Load integer to word dictionary
with open('kramer_int2word.pkl', 'rb') as file:
    int2word = pickle.load(file)

In [ ]:
# Load the codes
with open('codes.pkl', 'rb') as file:
    codes = pickle.load(file)

In [8]:
# Compare to similar output in "Process_Scripts" as a test
source_int[:5]

[[267, 1, 19, 57, 6],
 [1, 1035, 43, 28, 154, 13, 190, 680, 10],
 [491, 10],
 [1035, 13, 1035, 1035, 10],
 [1035, 6]]

### Something Useful to Say

In [ ]:
def model_inputs():
    '''Create palceholders for inputs to the model'''
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

    return input_data, targets, lr, keep_prob

In [ ]:
def process_encoding_input(target_data, vocab_to_int, batch_size):
    '''Remove the last word id from each batch and concat the <GO> to the begining of each batch'''
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return dec_input

In [ ]:
def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, sequence_length, attn_length):
    '''Create the encoding layer'''
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
    cell = tf.contrib.rnn.AttentionCellWrapper(drop, attn_length, state_is_tuple = True)
    enc_cell = tf.contrib.rnn.MultiRNNCell([cell] * num_layers)
    _, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw = enc_cell,
                                                   cell_bw = enc_cell,
                                                   sequence_length = sequence_length,
                                                   inputs = rnn_inputs, 
                                                   dtype=tf.float32)

    return enc_state

In [ ]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope,
                         output_fn, keep_prob):
    '''Decode the training data'''
    train_decoder_fn = tf.contrib.seq2seq.simple_decoder_fn_train(encoder_state)
    train_pred, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(
        dec_cell, train_decoder_fn, dec_embed_input, sequence_length, scope=decoding_scope)
    train_pred_drop = tf.nn.dropout(train_pred, keep_prob)
    return output_fn(train_pred_drop)

In [ ]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id,
                         maximum_length, vocab_size, decoding_scope, output_fn, keep_prob):
    '''Decode the prediction data'''
    infer_decoder_fn = tf.contrib.seq2seq.simple_decoder_fn_inference(
        output_fn, encoder_state, dec_embeddings, start_of_sequence_id, end_of_sequence_id, maximum_length, vocab_size)
    infer_logits, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell, infer_decoder_fn, scope=decoding_scope)
    return infer_logits

In [ ]:
def decoding_layer(dec_embed_input, dec_embeddings, encoder_state, vocab_size, sequence_length, rnn_size,
                   num_layers, vocab_to_int, keep_prob, attn_length):
    '''Create the decoding cell and input the parameters for the training and inference decoding layers'''
    
    with tf.variable_scope("decoding") as decoding_scope:
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
        cell = tf.contrib.rnn.AttentionCellWrapper(drop, attn_length, state_is_tuple = True)
        dec_cell = tf.contrib.rnn.MultiRNNCell([cell] * num_layers)
        
        weights = tf.truncated_normal_initializer(stddev = 0.1)
        biases = tf.zeros_initializer()
        output_fn = lambda x: tf.contrib.layers.fully_connected(x, 
                                                                vocab_size, 
                                                                None, 
                                                                scope=decoding_scope,
                                                                weights_initializer = weights,
                                                                biases_initializer = biases)

        train_logits = decoding_layer_train(
            encoder_state[0], dec_cell, dec_embed_input, sequence_length, decoding_scope, output_fn, keep_prob)
        decoding_scope.reuse_variables()
        infer_logits = decoding_layer_infer(encoder_state[0], dec_cell, dec_embeddings, vocab_to_int['<GO>'],
                                            vocab_to_int['<EOS>'], sequence_length, vocab_size,
                                            decoding_scope, output_fn, keep_prob)

    return train_logits, infer_logits

In [ ]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size, sequence_length, source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size, rnn_size, num_layers, vocab_to_int, attn_length):
    
    '''Use the previous functions to create the training and inference logits'''
    
    enc_embed_input = tf.contrib.layers.embed_sequence(input_data, source_vocab_size+1, enc_embedding_size)
    enc_state = encoding_layer(enc_embed_input, rnn_size, num_layers, keep_prob, sequence_length, attn_length)

    dec_input = process_encoding_input(target_data, vocab_to_int, batch_size)
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size+1, dec_embedding_size], -1.0, 1.0))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    train_logits, infer_logits = decoding_layer(dec_embed_input, dec_embeddings, enc_state, target_vocab_size+1, 
                                                sequence_length, rnn_size, num_layers, vocab_to_int, keep_prob, 
                                                attn_length)
    
    return train_logits, infer_logits

In [ ]:
# Set the parameters
epochs = 100
batch_size = 128
rnn_size = 512
num_layers = 2
encoding_embedding_size = 512
decoding_embedding_size = 512
attn_length = 10
learning_rate = 0.0005
keep_probability = 0.8

In [ ]:
train_graph = tf.Graph()
with train_graph.as_default():
    
    # Load the model inputs
    input_data, targets, lr, keep_prob = model_inputs()
    # Sequence length will be the max line length for each batch
    sequence_length = tf.placeholder_with_default(max_line_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)
    
    # Create the logits from the model
    train_logits, inference_logits = seq2seq_model(
        tf.reverse(input_data, [-1]), targets, keep_prob, batch_size, sequence_length, len(source_vocab_to_int), 
        len(target_vocab_to_int), encoding_embedding_size, decoding_embedding_size, rnn_size, num_layers, 
        target_vocab_to_int, attn_length)
    
    # Create a tensor to be used for making predictions.
    tf.identity(inference_logits, 'logits')
    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            train_logits,
            targets,
            tf.ones([input_shape[0], sequence_length]))

        # Optimizer
        optimizer = tf.train.AdamOptimizer(learning_rate)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)

In [ ]:
# Should this be moved up?
def pad_sentence_batch(sentence_batch, vocab_to_int):
    """Pad lines with <PAD> so each line of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]

In [ ]:
def batch_data(source, target, batch_size):
    """Batch source and target together"""
    for batch_i in range(0, len(source)//batch_size):
        start_i = batch_i * batch_size
        source_batch = source[start_i:start_i + batch_size]
        target_batch = target[start_i:start_i + batch_size]
        yield (np.array(pad_sentence_batch(source_batch, source_vocab_to_int)), 
               np.array(pad_sentence_batch(target_batch, target_vocab_to_int)))

In [ ]:
train_valid_split = int(len(source_int)*0.1)

train_source = source_int[train_valid_split:]
train_target = target_int[train_valid_split:]

valid_source = source_int[:train_valid_split]
valid_target = target_int[:train_valid_split]

print(len(train_source))
print(len(valid_source))

In [ ]:
learning_rate_decay = 0.95
display_step = 50
stop_early = 0
stop = 3
total_train_loss = 0
summary_valid_loss = []


checkpoint = "best_model.ckpt" 

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(1, epochs+1):
        for batch_i, (source_batch, target_batch) in enumerate(
                batch_data(train_source, train_target, batch_size)):
            start_time = time.time()
            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 sequence_length: target_batch.shape[1],
                 keep_prob: keep_probability})

            total_train_loss += loss
            end_time = time.time()
            batch_time = end_time - start_time
            
            if batch_i % display_step == 0:
                print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                      .format(epoch_i,
                              epochs, 
                              batch_i, 
                              len(train_source) // batch_size, 
                              total_train_loss / display_step, 
                              batch_time*display_step))
                total_train_loss = 0

            if batch_i % 235 == 0 and batch_i > 0:
                total_valid_loss = 0
                start_time = time.time()
                for batch_ii, (source_batch, target_batch) in \
                        enumerate(batch_data(valid_source, valid_target, batch_size)):
                    valid_loss = sess.run(
                    cost, {input_data: source_batch,
                           targets: target_batch,
                           lr: learning_rate,
                           sequence_length: target_batch.shape[1],
                           keep_prob: 1})
                    total_valid_loss += valid_loss
                end_time = time.time()
                batch_time = end_time - start_time
                avg_valid_loss = total_valid_loss / (len(valid_source) / batch_size)
                print('Valid Loss: {:>6.3f}, Seconds: {:>5.2f}'.format(avg_valid_loss, batch_time))
                
                learning_rate *= learning_rate_decay
                
                summary_valid_loss.append(avg_valid_loss)
                if avg_valid_loss <= min(summary_valid_loss):
                    print('New Record!') 
                    stop_early = 0
                    saver = tf.train.Saver() 
                    saver.save(sess, checkpoint)
                
                else:
                    print("No Improvement.")
                    stop_early += 1
                    if stop_early == stop:
                        break
        if stop_early == stop:
            print("Stopping Training.")
            break

In [ ]:
def sentence_to_seq(sentence, vocab_to_int):
    '''Prepare the predicted sentence for the model'''
    
    sentence = clean_text(sentence)
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in sentence.split()]

In [ ]:
# To create your own input sentence
#input_sentence = 'Oh my God they killed Kenny!'

# To use an sentence from the data
#random = np.random.choice(len(short_text))
#input_sentence = short_text[random]

# Clean the input sentence before it is used in the model
#input_sentence = sentence_to_seq(input_sentence, source_vocab_to_int)

#checkpoint = "./" + checkpoint 

#loaded_graph = tf.Graph()
#with tf.Session(graph=loaded_graph) as sess:
#    # Load the saved model
#    loader = tf.train.import_meta_graph(checkpoint + '.meta')
#    loader.restore(sess, checkpoint)
    
    # Load the tensors to be used as inputs
#    input_data = loaded_graph.get_tensor_by_name('input:0')
#    logits = loaded_graph.get_tensor_by_name('logits:0')
#    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
    
#    response_logits = sess.run(logits, {input_data: [input_sentence],keep_prob: 1.0})[0]

#print('Input')
#print('  Word Ids:      {}'.format([i for i in input_sentence]))
#print('  Input Words: {}'.format([source_int_to_vocab[i] for i in input_sentence]))

#print('\nResponse')
#print('  Word Ids:      {}'.format([i for i in np.argmax(response_logits, 1)]))
#print('  Response Words: {}'.format([target_int_to_vocab[i] for i in np.argmax(response_logits, 1)]))